# Orientation Estimation


## Reference

- https://towardsdatascience.com/matplotlib-animations-in-jupyter-notebook-4422e4f0e389
    
- https://matplotlib.org/stable/gallery/animation/simple_anim.html 

- https://seanng.my/2020/07/15/wireless-imu-sensor-using-bluetooth-low-energy-ble/

- https://roboticsclubiitk.github.io/2017/12/21/Beginners-Guide-to-IMU.html

- Complementary filter https://www.pieter-jan.com/node/11

- Arduino IMU https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_accelerometer

- https://stackoverflow.com/questions/23009549/roll-pitch-yaw-calculation

- https://habr.com/en/post/499190/

 - https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_magnetometer
 
 - https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_gyroscope
 
- https://www.infineon.com/dgdl/Infineon-3D_Magnetic_Sensor_for_Angle_Measurements-AN-v01_10-AN-v01_01-EN.pdf?fileId=5546d46265f064ff01665a3d22f055e3

![style=centerme](....png)

In [20]:
import matplotlib.pylab as plt
import numpy as np
import matplotlib.animation as animation
import pandas as pd
import time 
import collections
import serial
import threading

In [21]:
%pylab

Using matplotlib backend: MacOSX
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/Tommaso/miniconda3/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['std', 'Text', 'mean', 'plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [22]:
import numpy as np
#collections.deque(np.arange(1000), maxlen=1000),

Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 287, in process
    func(*args, **kwargs)
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/animation.py", line 909, in _start
    self._init_draw()
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/animation.py", line 1698, in _init_draw
    self._draw_frame(frame_data)
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/animation.py", line 1720, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_62926/3760398566.py", line 59, in animate
    Text.set_text(title)
TypeError: set_text() missing 1 required positional argument: 's'


In [23]:
class arduino(object):#serial_port,baud_rate=baud_rate):
    
    def __init__(self,serial_port='/dev/cu.usbmodem14201',baud_rate=9600,vector_lenght=500,alpha=0.02):
        
        self.serial_port   = serial_port
        self.baud_rate     = baud_rate
        
        self.alpha         = 0.02
        
        self.vector_lenght = vector_lenght
        
        
        self.raw_data = collections.deque([0]*vector_lenght, maxlen=vector_lenght)

    def read_serial_monitor(self,arduinoData):
    
        arduinoString = arduinoData.readline()
        arduinoString = arduinoString.decode("utf-8")
        arduinoString = arduinoString.strip()

        self.char_list     = arduinoString.split('\t')  
        

        return float(arduinoString)
   
    def read(self):
            
        self.arduinoData = serial.Serial(self.serial_port, self.baud_rate) 

        while True:
            x = self.read_serial_monitor(self.arduinoData)
            #print("{:>30}".format(x),end="")
            self.raw_data.append(x)
        
       
            

# Yaw Pitch Roll

In [34]:
def mean_update(mean,x,n):
    mean = (mean*n+x)/(n+1)
    n = n+1
    return mean,n

start_time = time.time()

my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    
fig, ax3 = plt.subplots(1,1,figsize=[6.4,4.8])

raw_data, = ax3.plot(my_arduino.raw_data,np.arange(500))

mean = np.mean(my_arduino.raw_data)
std = np.std(my_arduino.raw_data)
mean_bar, = ax3.plot([0,mean],[-20,-20],lw=5,marker='o')
std_bar,  = ax3.plot([mean-2*std,mean+2*std],[-10,-10],lw=5,marker='o')

#Text = ax3.text(0,0,my_arduino.Roll['Complementary'][-1])
Text = ax3.set_title("{:.2f}\n{:.0f} s".format(my_arduino.raw_data[-1],start_time))

#lim = [-np.pi*1.1,np.pi*1.1]
#ax1.set_ylim(lim);ax2.set_ylim(lim);ax3.set_ylim(lim)

ax3.set_xticks([-16,-5,0,5,16])
#ax3.set_xlim([-np.deg2rad(20),np.deg2rad(20)])
ax3.set_xlim([-20,20])
ax3.grid(True)


def animate(i,):
    
    vector = my_arduino.raw_data
    x = vector[-1]

    mean = np.mean(vector)
    std  = np.std(vector)

    time_from_start = time.time()-start_time
    #mean,n = mean_update(mean,x,n)
    
    #RollAccelerometer.set_xdata(my_arduino.Roll['Accelerometer'])
    #RollGyroscope.set_xdata(my_arduino.Roll['Gyroscope'])
    raw_data.set_xdata(vector)
    mean_bar.set_xdata([0,mean])
    std_bar.set_xdata([mean-2*std,mean+2*std])

    title = "{:.1f}".format(x)
    title += "\nmean: {:.1f} | std: {:.1f}".format(mean,std)
    title += "\n{:.0f} s".format(time_from_start)
    Text.set_text(title)
    

ani = animation.FuncAnimation(fig, animate, interval=200)


Exception in thread Thread-31:
Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_62926/2096052595.py", line 31, in read
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_62926/2096052595.py", line 17, in read_serial_monitor
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/serial/serialposix.py", line 595, in read
    raise SerialException(
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)
